# 保存した教材バックアップをリストアします
<HR>

保存してある教材のバックアップファイルを展開して、指定された位置に配置します。

### 1. リストアの準備

このNotebookを実行すると、ホームディレクトリ直下の下記のフォルダを、バックアップファイルの内容で上書きコピーします。<br/>
**上書きしたくないファイル/フォルダは、下記のフォルダから退避しておいて下さい。**
* textbook
  + 教材Notebookおよび教材Notebookから呼び出すスクリプトなど
* info
  + 受講者が参照する参考資料など
* draft
  + 教材以外で保存して置きたいNotebook、その他ファイル

### 2. リストアの作業用フォルダを作成します

* リストアするバックアップファルを格納して、展開するための作業用フォルダです。
* フォルダを変更する場合は、RESTORE_WORK_DIR を変更して下さい。
* フォルダが既存の場合、フォルダ配下のファイルは削除されます。必要があれば事前に退避して下さい。

**次のセルを実行して、作業用フォルダを作成して下さい。**

In [ ]:
RESTORE_WORK_DIR="$HOME/restore"

# create restore work directory
[ -e $RESTORE_WORK_DIR ] || {
  mkdir -p $RESTORE_WORK_DIR
  chown $USER:$USER $RESTORE_WORK_DIR
}

# creanlup restore work directory
rm -rf $RESTORE_WORK_DIR/*

### 3. バックアップファイルを転送します

* リストアする バックアップファイル(tar.gz 形式)を、上記で作成した作業用フォルダに転送して下さい。
* バックアップファイルがローカルマシンにある場合は、下記の手順でアップロードできます。
  1. ブラウザのHome画面で、リストア作業用フォルダを開きます。
  2. 画面右上の[Upload]ボタンをクリックします。ファイル選択ダイアログが開きます。
  3. ファイルを選択して[開く]ボタンをクリックします。 バックアップファイル名がリストされます。
  4. バックアップファイル名の行の右端に表示される[Upload]ボタンをクリックします。

### 4. バックアップファイル名を指定します

* ARCHIVE_FILE_NAME にリストアするバックアップファイル名を指定して下さい。
```
ARCHIVE_FILE_NAME="course_backup.tar.gz"
```
* バックアップファイルは、上のセルで作成した RESTORE_WORK_DIR に格納されているものとします。

** 次のセルでバックアップファイル名を指定して、セルを実行して下さい。**

In [ ]:
ARCHIVE_FILE_NAME=""

### 4. 次のセルを実行して、バックアップファイルを展開して下さい


バックアップファイルを展開して、ホームディレクトリに配置します。

In [ ]:
# check setting
do_restore=1
[ -z "$ARCHIVE_FILE_NAME" ]  && {
    echo "ARCHIVE_FILE_NAME is not set."
    do_restore=0
}

ARCHIVE_FILE_PATH=$RESTORE_WORK_DIR/$ARCHIVE_FILE_NAME
if [ "$do_restore" -eq 1 ] && [ ! -e "$ARCHIVE_FILE_PATH" ]; then
    echo "ARCHIVE_FILE is not found."
    do_restore=0
fi

if [ "$do_restore" -eq 1 ]; then
    # expand the backup archive file 
    (cd $RESTORE_WORK_DIR; tar xvfz $ARCHIVE_FILE_NAME)
    chown -R $USER:$USER $RESTORE_WORK_DIR

    # list of files/folders to move
    move_list=""
    SKIPS_REG="\.$|\.\.$|$ARCHIVE_FILE_NAME"
    for item in $(ls -a $RESTORE_WORK_DIR); do
        [[ "$item" =~ $SKIPS_REG ]] && continue
        move_list=$move_list" $item"
    done

    # move files/folders
    for target in $move_list; do
        cp -r $RESTORE_WORK_DIR/$target "$HOME/."
    done
fi

# delete restore work directory
rm -rf $RESTORE_WORK_DIR

echo
echo "... backup finished."